In [9]:
import requests
from openai import OpenAI
import pandas as pd
import re

In [10]:
df = pd.read_json('math_qa.json')

questions = df['Q'].tolist()
answers = df['A'].tolist()
pages = df['page'].tolist()

print("Questions:", questions[0])
print("Answers:", answers[0])
print("page:", pages[0])
print(len(questions))

Questions: Q: What is the significance of "infinite descent" in number theory, particularly in proving results related to Diophantine equations?  

Answers:   
A: The significance of "infinite descent" in number theory lies in its utility for establishing the non-existence of solutions to certain Diophantine equations by demonstrating that any proposed solution leads to a smaller solution, thus creating an infinite regression that contradicts the notion of a smallest natural number.
page: 1
300


In [11]:
from sklearn.metrics import f1_score

def calculate_f1(string1, string2):
    set1 = set(string1.split())
    set2 = set(string2.split())
    tp = len(set1 & set2)
    precision = tp / len(set2) if set2 else 0
    recall = tp / len(set1) if set1 else 0
    if precision + recall == 0:
        f1 = 0
    else:
        f1 = 2 * (precision * recall) / (precision + recall)
    return f1

In [12]:
url = "http://i9ubuntu.eason.best:5020/search"

headers = {
    "Content-Type": "application/json",
}
Error_L = []
F1_L = []
Errors = 0
page_check = 0
tokens = 0
for i, q in enumerate(questions):
    queryN = q
    payload = {
    "query": queryN
    }
    # response = requests.post(url, headers=headers, json=payload)
    # print(response.status_code)
    # pattern = r'"pages":\[(.*?)\]'
    # matches = re.findall(pattern, response.text)
    # contains_page = [match for match in matches if str(pages[i]) in match.split(', ') if match.strip()]
    # print(contains_page)
    # if len(contains_page) != 0:
    #     page_check += 1
    # print(pages[i], page_check)
    try:
        client = OpenAI(api_key='OPENAI_API_KEY_PLACEHOLDER')
        r = client.chat.completions.create(
            messages=[
                {
                "role": "system", "content": "You are a math expert",
                "content": "user", "content": "I am giving you a question. Provide a short answer for the question. Say nothing else." + ", Question:" + q
                }
            ],
            model="gpt-4o-mini",
        )
        res = r.choices[0].message.content
        f1 = calculate_f1(answers[i], res)
        print(f1)
        F1_L.append(f1)
        usage = r.usage
        tokens += usage.total_tokens
        print(tokens)

    except Exception as e:
        Errors += 1
        Error_L.append(i)
        print(e)

    print("--------") 
    if i >= 100:
        break

0.39506172839506176
117
--------
0.16666666666666669
192
--------
0.4
262
--------
0.5671641791044776
342
--------
0.5454545454545454
443
--------
0.6176470588235294
561
--------
0.27999999999999997
632
--------
0.06896551724137931
701
--------
0.4032258064516129
1046
--------
0.6857142857142857
1122
--------
0.3333333333333333
1193
--------
0.186046511627907
1267
--------
0.2962962962962963
1353
--------
0.46428571428571425
1444
--------
0.18421052631578946
1586
--------
0.6428571428571429
1651
--------
0.8571428571428572
1718
--------
0.4556962025316456
1834
--------
0.08
1886
--------
0.41379310344827586
1958
--------
0.8666666666666667
2052
--------
0.962962962962963
2153
--------
0.6875
2270
--------
0.425
2418
--------
0.43478260869565216
2513
--------
0.6923076923076923
2598
--------
0.43749999999999994
2700
--------
0.782608695652174
2795
--------
0.5581395348837209
2903
--------
0.8421052631578948
3027
--------
0.3157894736842105
3113
--------
0.3728813559322034
3203
--------


In [13]:
print(sum(F1_L) / len(F1_L))
print(tokens)

0.45763681079827623
13140
